In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import re

# Configurar Selenium
options = Options()
options.headless = False  # Mostrar el navegador para evitar bloqueos
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")  # Ocultar Selenium
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# Iniciar WebDriver
service = Service()
driver = webdriver.Chrome(options=options)

rickymorty = "https://www.imdb.com/title/tt2861424/episodes/"
driver.get(rickymorty)

# Esperar para simular interacción humana
time.sleep(5)  # Aumentar si sigue bloqueando

# Simular movimiento del mouse
actions = ActionChains(driver)
actions.move_by_offset(100, 100).perform()

# Obtener HTML después de esperar y analizar con BeautifulSoup
from bs4 import BeautifulSoup as bs
soup = bs(driver.page_source, "html.parser")

driver.quit() # Cerrar navegador

# Extraer títulos y calificaciones
titulo = soup.find_all(class_= "sc-f2169d65-4 kBjDMi")
calificaciones = soup.find_all(class_="sc-e2dbc1a3-0 jeHPdh sc-ac2c9267-3 eNyVgC")

# Itera a través de los elementos y obtén el texto
for title, calif in zip(titulo, calificaciones):
    titulo_texto = title.text.strip()  # Limpia espacios innecesarios

    # 🛠 Extraer "T1.E1" usando regex
    match = re.search(r"(T\d+\.E\d+)", titulo_texto)  
    if not match:
        print(f"⚠️ No se encontró formato en: {titulo_texto}")
        continue  # Salta a la siguiente iteración si no encuentra el formato esperado

    cap = match.group(1)  # Extrae el "T1.E1"
    
    # 🛠 Normalizar formato: S01E01
    cap_parts = cap.split(".E")
    season = "S" + cap_parts[0][1:].zfill(2)  # Convierte "T1" en "S01"
    episode = "E" + cap_parts[1].zfill(2)  # Asegura que "E1" se convierta en "E01"
    cap = season + episode

    # 🛠 Extraer calificación
    stars = calif.text.split()[0]  # Primer número antes de "/10"
    votes_match = re.search(r"\((\d[\d,.]*\s*[kK]?)\)", calif.text)
    votes = re.sub(r"\s+", "", votes_match.group(1)).upper() if votes_match else "0" # Extrae votos y elimina espacios

    capituloycalificación = f"{cap} - {stars} - {votes}"

    print(capituloycalificación)



S01E01 - 7.9/10 - 18K
S01E02 - 8.6/10 - 18K
S01E03 - 8.3/10 - 17K
S01E04 - 8.6/10 - 17K
S01E05 - 9.0/10 - 18K
S01E06 - 9.1/10 - 18K
S01E07 - 7.9/10 - 15K
S01E08 - 8.7/10 - 17K
S01E09 - 8.4/10 - 15K
S01E10 - 9.3/10 - 19K
S01E11 - 8.4/10 - 15K


In [4]:
# Ahora una extracción a todas las temporadas de relevacncia

import pandas as pd # Con el fin de ordenar en un dataframe

service = Service()
driver = webdriver.Chrome(options=options)  # Solo se inicializa una vez

# 🔗 Base URL
base_url = "https://www.imdb.com/title/tt2861424/episodes"
max_season = 7
capituloycalificación = []

# 🔄 Iterar por cada temporada
for season_number in range(1, max_season + 1):
    current_url = f"{base_url}/?season={season_number}&ref_=ttep"
    driver.get(current_url)
    time.sleep(5)  # Esperar para evitar bloqueos

    # 🖱️ Simular movimiento del mouse
    actions = ActionChains(driver)
    actions.move_by_offset(100, 100).perform()

    # 📄 Obtener HTML y analizar con BeautifulSoup
    soup = bs(driver.page_source, "html.parser")

    # 📝 Extraer títulos y calificaciones
    titulo = soup.find_all(class_="sc-f2169d65-4 kBjDMi")
    calificaciones = soup.find_all(class_="sc-e2dbc1a3-0 jeHPdh sc-ac2c9267-3 eNyVgC")

    if not titulo:
        break  # Si no hay episodios, salimos del bucle

    # 🔄 Iterar episodios de la temporada actual
    for title, calif in zip(titulo, calificaciones):
        titulo_texto = title.text.strip()

        # 🛠 Extraer "T1.E1" con regex
        match = re.search(r"(T\d+\.E\d+)", titulo_texto)
        if not match:
            print(f"⚠️ No se encontró formato en: {titulo_texto}")
            continue  

        cap_parts = match.group(1).split(".E")
        season = "S" + cap_parts[0][1:].zfill(2)  
        episode = "E" + cap_parts[1].zfill(2)  
        cap = season + episode

        # 🛠 Extraer calificación y votos
        stars = calif.text.split()[0]  
        votes_match = re.search(r"\((\d[\d,.]*\s*[kK]?)\)", calif.text)
        votes = re.sub(r"\s+", "", votes_match.group(1)).upper() if votes_match else "0"

        # 📌 Guardar resultado
        capituloycalificación.append((cap, stars, votes))
        #print(f"{cap} - {stars} - {votes}")

# ❌ Cerrar el navegador al final
driver.quit()

# 📊 Guardar en DataFrame
rating = pd.DataFrame(capituloycalificación, columns=["Cap-Seasons", "Stars", "Votes"])
print(rating)


   Cap-Seasons   Stars Votes
0       S01E01  7.9/10   18K
1       S01E02  8.6/10   18K
2       S01E03  8.3/10   17K
3       S01E04  8.6/10   17K
4       S01E05  9.0/10   18K
..         ...     ...   ...
66      S07E06  7.8/10  6.4K
67      S07E07  6.4/10  6.2K
68      S07E08  4.5/10  8.7K
69      S07E09  7.9/10  6.1K
70      S07E10  9.4/10  9.7K

[71 rows x 3 columns]
